In [1]:
## Import typical data science packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [2]:
## Import and concatenate the data files

# Directory containing the CSV files
data_dir = r"../Data/"
# Filenames to be processed
all_files = [
    "adm2014_rv.csv",
    "adm2015_rv.csv",
    "adm2016_rv.csv",
    "adm2017_rv.csv",
    "adm2018_rv.csv",
    "adm2019_rv.csv",
    "adm2020_rv.csv",
    "adm2021_rv.csv",
    "adm2022_rv.csv",
    "adm2023.csv"
]

# Read and combine all matched CSV files
dfs = []
for file in all_files:
    # Construct full file path
    file_path = os.path.join(data_dir, file)
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Extract the four digits in the filename

    match = re.search(r'(\d{4})', file)
    if match:
        df['year'] = match.group(1)
    else:
        df['year'] = None
    # Clean white space out of column names
    df.columns = df.columns.str.strip()
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

In [3]:
# Quick description of the combined data
combined_df.describe()

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,...,ENRLUN,ENRLFTAN,ENRLFTUN,ENRLPTAN,ENRLPTUN,SATVR50,SATMT50,ACTCM50,ACTEN50,ACTMT50
count,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,20575.000000,...,3524.000000,287.000000,3512.000000,213.000000,2672.000000,2004.000000,2004.000000,1977.000000,1862.000000,1864.000000
mean,225924.798104,1.521069,2.825176,1.235188,2.260267,2.323111,2.849623,2.507315,1.663913,2.809380,...,0.651249,13.533101,0.611617,0.422535,0.055015,589.972555,581.571856,24.581184,24.212137,23.071352
std,106323.327450,1.033901,1.116830,0.760811,1.158216,1.277863,0.869616,1.537558,1.067812,0.760858,...,7.202760,19.960401,6.980100,2.540216,0.572329,71.467100,79.225856,4.567600,5.196783,4.429292
min,100654.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,380.000000,310.000000,5.000000,2.000000,5.000000
25%,159391.000000,1.000000,2.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,3.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,540.000000,530.000000,21.000000,21.000000,20.000000
50%,197036.000000,1.000000,3.000000,1.000000,2.000000,2.000000,3.000000,2.000000,1.000000,3.000000,...,0.000000,5.000000,0.000000,0.000000,0.000000,580.000000,570.000000,24.000000,23.000000,23.000000
75%,231411.000000,2.000000,3.000000,1.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,...,0.000000,15.000000,0.000000,0.000000,0.000000,635.000000,620.000000,28.000000,27.750000,26.000000
max,499635.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,9.000000,...,358.000000,158.000000,347.000000,35.000000,18.000000,760.000000,800.000000,35.000000,35.000000,35.000000


In [4]:
# Do basic cleaning of the combined data
# Drop duplicates
combined_df.drop_duplicates(inplace=True)
# Drop rows with missing UNITID
combined_df.dropna(subset=['UNITID'], inplace=True)
# Ensure UNITID is integer
combined_df['UNITID'] = combined_df['UNITID'].astype(int)
# Reset index after cleaning
combined_df.reset_index(drop=True, inplace=True)

In [5]:
## Look at which observations are not recorded at all in certain years

# For each column, find years where all entries are null for that column
null_years = {}
for col in combined_df.columns:
    # Group by year and check if all values in the column are null for each year
    years_all_null = combined_df.groupby('year')[col].apply(lambda x: x.isnull().all())
    # Get years where all values are null
    null_years[col] = years_all_null[years_all_null].index.tolist()

# Display columns with at least one year where all entries are null
for col, years in null_years.items():
    if years:
        print(f"Column '{col}' is all null in years: {years}")

Column 'XSATWR25' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'SATWR25' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'XSATWR75' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'SATWR75' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'XACTWR25' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'ACTWR25' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'XACTWR75' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'ACTWR75' is all null in years: ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
Column 'ADMCON10' is all null in years: ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
Column 'ADMCON11' is all null in years: ['2014', '2015', '2

In [11]:
## Import and concatenate the characteristic files

# Directory containing the CSV files
# Filenames to be processed
all_char_files = [
    "../Data/hd2014.csv",
    "../Data/hd2015.csv",
    "../Data/hd2016.csv",
    "../Data/hd2017.csv",
    "../Data/hd2018.csv",
    "../Data/hd2019.csv",
    "../Data/hd2020.csv",
    "../Data/hd2021.csv",
    "../Data/hd2022.csv",
    "../Data/HD2023.csv"
]

# Read and combine all matched CSV files
dfs2 = []
for file in all_char_files:
    # Read the CSV file
    df2 = pd.read_csv(file, low_memory=False, encoding='latin1')
    # Extract the four digits in the filename
    match = re.search(r'(\d{4})', file)
    if match:
        df2['year'] = match.group(1)
    else:
        df2['year'] = None
    # Clean white space out of column names
    df2.columns = df2.columns.str.strip()
    dfs2.append(df2)

characteristics = pd.concat(dfs2, ignore_index=True)

In [13]:
# Clean the characteristics data
characteristics.drop_duplicates(inplace=True)
characteristics.dropna(subset=['UNITID','year'], inplace=True)
characteristics['UNITID'] = characteristics['UNITID'].astype(int)
characteristics.reset_index(drop=True, inplace=True)

In [20]:
# Left merge the characteristics data onto the combined admissions data on 'UNITID'
merged_df = pd.merge(combined_df, characteristics, on=['UNITID', 'year'], how='left', indicator=True)

In [21]:
merged_df.head()

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,...,C18SZSET,C21BASIC,C21IPUG,C21IPGRD,C21UGPRF,C21ENPRF,C21SZSET,UEIS,ï»¿UNITID,_merge
0,100654,1,2,1,2,3,2,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,100663,1,3,1,1,3,3,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,100706,1,2,1,1,3,2,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,100724,1,3,2,3,3,3,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,100751,1,2,1,1,3,3,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [23]:
# Show how many rows were matched and how many were not
merged_df["_merge"].value_counts()

# Drop unmatched rows and the merge indicator column
merged_df = merged_df[merged_df["_merge"] == "both"].drop(columns=["_merge"])

In [24]:
# Save the final combined DataFrame to a CSV file
merged_df.to_csv('../Data/combined_admissions_characteristics.csv', index=False)